In [1]:
import matplotlib.pyplot as plt
from graphics import *
from operator import *
from random import *
import numpy as np
import timeit

In [2]:
"""def initialize(s = 1, N = 100, width = 1000, height = 1000):

    Initializes our agent set with randomly directed speeds, draws the window and the agents

    agents = [Point(width*random(), height*random()) for i in range(N)]
    speeds = [np.array([0.0, 0.0]) for i in range(N)]
    speed_arrows = N * [0]
    for i in range(N):
        theta = 2 * np.pi * random()
        speeds[i][0] = s * np.cos(theta)
        speeds[i][1] = s * np.sin(theta)

        point1 = agents[i]
        norm_speed = speeds[i]/s
        x_end = point1.getX() + norm_speed[0]
        y_end = point1.getY() + norm_speed[1]
        point2 = Point(x_end, y_end)
        speed_arrows[i] = Line(point1, point2)
        speed_arrows[i].setArrow("last")
    
    return agents, speeds, draw_agents(agents, speed_arrows, width, height, N)

    
def draw_agents(agents, speeds, width, height, N):
    win = GraphWin("Swarm", width, height) # size of box
    for i in range(N):
        agents[i].draw(win)
        speeds[i].draw(win)
        
    win.getMouse()
    return win
"""

def initialize(s = 1, N = 50, width = 1000, height = 1000):
    """
    Initializes our agent set with randomly directed speeds, draws the window and the agents
    """
    locations = [Point(width*random(), height*random()) for i in range(N)]
    speeds = [np.array([0.0, 0.0]) for i in range(N)]
    agents = N * [0]
    for i in range(N):
        theta = 2 * np.pi * random()
        speeds[i][0] = s * np.cos(theta)
        speeds[i][1] = s * np.sin(theta)

        point1 = locations[i]
        norm_speed = speeds[i]/s
        x_end = point1.getX() + norm_speed[0]
        y_end = point1.getY() + norm_speed[1]
        point2 = Point(x_end, y_end)
        agents[i] = Line(point1, point2)
        agents[i].setArrow("last")
        print(agents[i].getP1())
    
    return agents, speeds, draw_agents(agents, width, height, N)


def draw_agents(agents, width, height, N):
    win = GraphWin("Swarm", width, height) # size of box
    for i in range(N):
        agents[i].draw(win)
        
    win.getMouse()
    return win


def couple_speeds(agents, speeds, a, s, N):
    """
    Simplest model: for each agent, it will give its nearest neighbour a fraction of its speed and re-normalize it
    
    """
    newspeeds = [np.array([0.0, 0.0]) for i in range(N)]
    nearest_neighbours = [nearest_neighbour(agent, agents, N) for agent in agents]
    for i in range(N):
            weightedSpeed = a * speeds[nearest_neighbours[i]]
            newspeeds[i] = speeds[i] + weightedSpeed
            newspeeds[i] = s * normalized(speeds[i]) ### PROBLEMA?-------------------------------------------------

    return newspeeds

def get_distances(agent, agents, N):
    """
    Given one angent and the set of all agents, 
    computes the distances from the first to all of the others
    """
    x1 = agent.getP1().getX()
    y1 = agent.getP1().getY()
    dists = N * [0]
    for i in range(N):
        x2 = agents[i].getP1().getX()
        y2 = agents[i].getP1().getY()
        dists[i] = (x2 - x1)**2 + (y2 - y1)**2

        if dists[i] == 0:
            dists[i] = 0.1
    return dists


def nearest_neighbour(agent, agents, N):
    """
    Returns the index for the agent with smallest Eucledian distance to agent in question
    """
    distances = get_distances(agent, agents, N)
    j = next(i for i in range(N) if agents[i] is agent)
    distances[j] = distances[j-1] + 1

    return distances.index(min(distances))


def treat_boundary(x_bound, y_bound, agents, speeds, N):
    for i in range(N):
        [dx, dy] = [0, 0]
        [x, y] = [agents[i].getP1().getX(), agents[i].getP1().getY()]
        if x > x_bound:
            speeds[i][0] = -speeds[i][0]
            dx = x_bound - x         
        
        elif x < 0:
            speeds[i][0] = -speeds[i][0]
            dx = -x
        
        if y > y_bound:
            speeds[i][1] = -speeds[i][1]
            dy = y_bound - y
        
        elif y < 0:
            speeds[i][1] = -speeds[i][1]
            dy = -y

        agents[i].move(dx, dy)
        
        
def periodic_boundary(x_bound, y_bound, agents, speeds, N):  #Changed from rigid boundaries do periodic boundary condition
    [dx, dy] = [0, 0]
    for i in range(N):
        [x, y] = [agents[i].getP1().getX(), agents[i].getP1().getY()]
        if x > x_bound:
            dx = -x_bound         
        
        elif x < 0:
            dx = x_bound
        
        if y > y_bound:
            dy = -y_bound
        
        elif y < 0:
            dy = y_bound

        agents[i].move(dx, dy)

def next_step(agents, speeds, dt, N):
    dxvec = [dt * speeds[i][0] for i in range(N)]
    dyvec = [dt * speeds[i][1] for i in range(N)]
    for i in range(N):
        agents[i].move(dxvec[i], dyvec[i])
    return


def update_speeds(agents, newspeeds, window, N):
    for i in range(N):
        point1 = agents[i].getP1()
        x_end = point1.getX() + newspeeds[i][0]
        y_end = point1.getY() + newspeeds[i][1]
        point2 = Point(x_end, y_end)
        newagent = Line(point1, point2)
        newagent.setArrow('last')
        agents[i].undraw()
        newagent.draw(window)
        agents[i] = newagent
    return agents

        
def normalized(vector):
    if vector[0] == vector[1] == 0:
        return vector
    return vector / np.linalg.norm(vector)


#def get_v_avg(speeds, N):
#    return
#   
#def get_potencials(distances, N):
#    potentials = np.zeros(N, N)
#    for i in range(N):
#        for j in range(i+1,N):
#            potentials[i][j] = potentials[j][i] = morse_pot(distances[i][j])
#    return potentials  
#    
#def morse_pot(r, ca = 0.5, cr = 1, la = 3, lr = 0.5):
#    return cr*np.exp(-r/lr) - ca * np.exp(-r)
#
#def weird_paper(agents, N, dt):
#    distances = [get_distances(agent, agents) for agent in agents]
#    potencials = get_potencials(distances, N)
#    for i in range(N):
#            newSpeed = speed[i] + dt * potentials
#            speeds[i] = speeds[i] + weightedSpeed
#            speeds[i] = s * normalized(speeds[i]) ### PROBLEMA?-------------------------------------------------
#
#    return

In [3]:
def couzin(agents, speeds, N, s, rr=1, ro=2, ra=3):
    # watch only particles inrepuls
    newspeeds = [np.array([0.0, 0.0]) for i in range(N)]
    for i in range(N): ### FIX - Eliminate, in some way the i-i interaction
        curr_agent = agents[i]
        distances = get_distances(curr_agent, agents, N)
        r_dir = np.array([0.0, 0.0])
        o_dir = np.array([0.0, 0.0])
        a_dir = np.array([0.0, 0.0])
        repulsion_flag = False    
        
        for j in range(N):
            if i == j:
                continue

            if distances[j] < rr:
                temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                temp_vec = normalized(temp_vec)
                r_dir = r_dir + temp_vec
                repulsion_flag = True
            
            elif not repulsion_flag:

                if distances[j] < ro:
                    o_dir = o_dir + speeds[j]
                
                elif distances[j] < ra:
                    temp_vec = np.array([agents[j].getX() - curr_agent.getX(), agents[j].getY() - curr_agent.getY()])
                    temp_vec = normalized(temp_vec)
                    a_dir = a_dir + temp_vec
            
                    
        if repulsion_flag:
            tot_dir =  normalized(-r_dir)
        else:
            o_dir = normalized(o_dir)
            a_dir = normalized(a_dir)
            tot_dir = o_dir + a_dir
            tot_dir = normalized(tot_dir)

        if np.linalg.norm(tot_dir) != 0:
            newspeeds[i] = s*tot_dir 

    return newspeeds
            

def vicsek(agents, speeds, N, s, noise, r): # s=speed, noise= letter csi temperature factor, r=radius of interaction
    # consider only particles within 'r' from pt_i, align pt_i with v_avg
    newspeeds = [np.array([0.0, 0.0]) for i in range(N)]
    for i in range(N):
        distances = get_distances(agents[i], agents, N)
        o_dir = np.array([0.0, 0.0])
        
        for j in range(N):
            if distances[j] < r:
                o_dir = o_dir + speeds[j]

        o_dir = s * normalized(normalized(o_dir) + noise * np.array([(-1)+2*random(),(-1)+2*random()]))
        if np.linalg.norm(o_dir) != 0:
            newspeeds[i] = o_dir
    
    return newspeeds

def gueron():
    return

In [4]:
def simulate(N_steps, a, dt, N, width, height, s):
    """
    Simulates motion of swarm. Recieves following parameters:

    N_steps  - number of steps to perform
    a -  coupling between neighbouring points
    dt - time step to be used 
    N - number of points to be used
    width & heigth - dimensions of window
    s - module of speed throughout agents

    """

    agents, speeds, window = initialize(s, N, width, height)
    for i in range(N_steps):
        next_step(agents, speeds, dt, N)
        import timeit

        start = timeit.default_timer()
        ## MODEL
        #newspeeds = vicsek(agents, speeds, N, s, noise = 0.9, r = 500)
        #newspeeds = couzin(agents, speeds, N, s,10,1900,2000)
        newspeeds = couple_speeds(agents, speeds, a, s, N)
        agents = update_speeds(agents, newspeeds, window, N)
        
        ## BOUNDARY CONDITIONS
        #treat_boundary(width, height, agents, speeds, N)
        periodic_boundary(width, height, agents, speeds, N)
        #print(v_avg)
        #v_avg = get_v_avg(speeds)
        #time.sleep(0.02)
        stop = timeit.default_timer()
        print stop - start
    window.close()

simulate(200, 0.1, 1, 50, 500, 500, 2)

Point(45.4727411618, 389.99989482)
Point(9.98268174263, 265.04421704)
Point(81.8139340544, 11.57842291)
Point(98.9861024119, 330.291367267)
Point(258.090160743, 259.941574366)
Point(482.019435392, 266.834398611)
Point(49.5595360443, 402.377515483)
Point(474.759542354, 200.832421562)
Point(84.0591639854, 458.45343084)
Point(29.408498318, 431.156978808)
Point(162.716814486, 13.9908558493)
Point(338.659855184, 280.220089873)
Point(130.347170728, 288.86345987)
Point(245.262752631, 26.2920138332)
Point(329.536062572, 488.703018458)
Point(373.097505512, 200.657670264)
Point(229.617098499, 418.511207324)
Point(444.661839239, 318.631257525)
Point(460.364690076, 476.944123339)
Point(71.7925273188, 86.6762249036)
Point(129.475783751, 259.874366174)
Point(288.165692583, 101.39335139)
Point(147.693925744, 296.232743151)
Point(357.805943186, 347.419456113)
Point(345.442200139, 172.120250687)
Point(379.302072957, 452.13784729)
Point(166.148405155, 462.180583143)
Point(40.0964605271, 128.769766347)
P